In [419]:
import random
import fitz
import re
import json
import pandas as pd

In [ ]:
def load_data(path):
    # Load intents
    with open(path, 'r') as file:
        intents = json.load(file)['intents']

    intents_dict = {}
    response_dict = {}
    for intent in intents:
        intents_dict[intent['tag']] = intent['patterns']
        response_dict[intent['tag']] = intent['responses']

    return intents_dict, response_dict

In [420]:
intents_dict, response_dict = load_data('../../raw_data/intents.json')
n_categories = len(intents_dict)
print(n_categories)

14


# Programmatically expand intents sections, starting with greeting

thesaurus for ease of access to synonyms

In [421]:
# load the data
pdf_path = "../raw_data/thesaurus.pdf"
doc = fitz.open(pdf_path)
text = ""
for page in doc:
    text += page.get_text("text")

Dont need intro text

In [422]:
trimmed_text = text[text.find("aback"):]
print(trimmed_text[:100])

aback [adv] taken unawares confused, surprised,
thrown off, thrown off guard*; concept 403
abaft [ad


In [423]:
# Regex pattern to match the word, the middle part including square brackets, and the definition
start_pattern = r"(\w+)\s\[\w+\d?\]\s(.+)"
end_pattern = r";? ?concepts? \d{1,3}"
lines = trimmed_text.split("\n")
start_text = ""
text_cleaned = []

for i in range(len(lines)):
    start_match = re.match(start_pattern, lines[i])
    if start_match:
        word, start_text = start_match.groups()

        for j in range(i+1, len(lines)):
            end_match = re.search(end_pattern, lines[j])
            if end_match:
                start_text += re.split(end_pattern, lines[j])[0]
                i = j
                break
            else:
                start_text += lines[j]

        text_cleaned.append((word, start_text))

In [424]:
# look for word in the text
def find_syns(word_to_find, context):
    syns = []
    for word, definition in text_cleaned:
        if word == word_to_find and context in definition:
            possible_responses = definition.split(",")
            for response in possible_responses:
                syns.append(response.strip())

    return syns

# gets all occurences of the word "good". Not every "good" works for our context, so we add some (context, that is).
# For now, i'll just manually edit it.
print(find_syns("good", "excellent"))

['pleasant', 'ﬁne acceptable', 'ace*', 'admirable', 'agreeable', 'bad', 'boss*', 'bully', 'capital', 'choice', 'commendable', 'congenial', 'crack*', 'deluxe', 'excellent', 'exceptional', 'favorable', 'ﬁrst-class', 'ﬁrst-rate', 'gnarly*', 'gratifying', 'great', 'honorable', 'marvelous', 'neat*', 'nice', 'pleasing', 'positive', 'precious', 'prime', 'rad*', 'recherché*', 'reputable', 'satisfactory', 'satisfying', 'select', 'ship-shape*', 'sound', 'spanking*', 'splendid', 'sterling', 'stupendous', 'super', 'superb', 'super-eminent', 'super-excellent', 'superior', 'tip-top*', 'up to snuff*', 'valuable', 'welcome', 'wonderful', 'worthy;']


# adding greeting intents

Get rid of synonyms that don't make sense for our purposes. Copied from above and manually edited. Prepare lists for forming greeting phrases and responses.

In [425]:
import itertools

greetings = ["Hello", "Hi", "Hey", "Howdy", "Greetings", "Good morning", "Good afternoon", "Good evening", "Hi there", "Hello there"]
additional = ["how are you?", "how are you doing?", "how's it going?", "what's up?", "how do you do?", ""]
combinations = list(itertools.product(greetings, additional))
combinations_joined = [', '.join(combination) if combination[1] != "" else combination[0] for combination in combinations]
print(len(combinations_joined))

60


In [426]:
responses = ["Well hello", "Hi there", "Hey yourself", "Howdy there", "Greetings", "Good morning", "Good afternoon", "Good evening", "Hi", "Hello"]
help_inquire = "How may I help you today?" # for conciseness
responses_additional = ["I'm doing well, thank you.", "I'm doing well, thanks for asking.", "It's going well, thank you.", "Not much, thanks for asking.",
                         "I'm doing well, thank you.", ". "]
responses_combined = list(itertools.product(responses, responses_additional))
responses_joined = ['. '.join(combination) if combination[1] != ". " else ''.join(combination) for combination in responses_combined]
responses_with_inquiry = [response  + help_inquire for response in responses_joined]
print(len(responses_joined))

60


Save updated intents

In [427]:
intents_dict['greeting'] = combinations_joined
response_dict['greeting'] = responses_joined

# adding goodbye intents

In [428]:
find_syns("goodbye", "farewell")

['farewell statement adieu',
 'adios',
 'bye-bye',
 'cheerio',
 'ciao',
 'godspeed*',
 'leave-tak-ing',
 'parting',
 'so long*',
 'swan song*',
 'toodle-oo*;']

In [429]:
goodbye_syns = find_syns("goodbye", "farewell")
goodbye_syns.remove("leave-tak-ing")
goodbye_syns.remove("godspeed*") # debated on keeping this
goodbye_syns.remove("swan song*")
goodbye_syns.remove("toodle-oo*;")
goodbye_syns.remove("parting")
print(goodbye_syns)

['farewell statement adieu', 'adios', 'bye-bye', 'cheerio', 'ciao', 'so long*']


In [430]:
goodbye_syns[0] = goodbye_syns[0].split(" ")[0] # remove the extra words from the first one
# remove '*' from the end of the words
goodbye_syns = [word.strip("*") for word in goodbye_syns]
print(goodbye_syns)

['farewell', 'adios', 'bye-bye', 'cheerio', 'ciao', 'so long']


In [431]:
# for correct grammar
def a_or_an(word):
    if word in ["excellent", "amazing", "outstanding", "exceptional"]:
        return "an " + word
    else:
        return "a " + word

good_syns = ["good", "great", "excellent", "amazing", "fantastic", "superb", "terrific", "outstanding", "exceptional", "marvelous", "splendid", "wonderful"]

goodbye_templates = []
response_templates = []
goodbye_syns.append("goodbye")
goodbye_syns.append("till next time")
goodbye_syns.append("see you later")
goodbye_syns.append("bye")

times = ["one", "day", "evening", "night", "rest of your day", "rest of your night", "rest of your evening", "rest of your morning"]

goodbye_additional = ["Thanks for your help.", "Thanks again.", "Thanks for helping.", "Take care."]
for syn in good_syns:
    for time in times:
        goodbye_additional.append(f"Have {a_or_an(syn)} {time}.")
    goodbye_additional.append(f"You've been {a_or_an(syn)} help.")

goodbye_additional.append("")

In [432]:
i = 0
for gb_syn in goodbye_syns:
    for gb_add in goodbye_additional:
        goodbye_templates.append(f"{gb_syn}{random.choice(['!','.'])} {gb_add}")
        if i < 3: # first three are thank yous
            response_templates.append("It's my pleasure. I'm glad I could be of assistance. Goodbye!")
            i += 1
        else:
            if gb_add == "":
                response_templates.append("Goodbye! Have a great day!") # just a goodbye
            else:
                response_templates.append("Goodbye! Please come again!")

In [433]:
intents_dict['goodbye'] = goodbye_templates
response_dict['goodbye'] = response_templates

# Adding thanks intents

In [434]:
print(intents_dict['thanks'])

['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me']


In [435]:
print(response_dict['thanks'])

['Happy to help!', 'Any time!', 'My pleasure']


In [436]:
thanks = ['thank you very much', 'thanks a lot', 'thank you so much', 'thanks so much', 'thank you kindly', 'thank you for that.']
thanks = thanks + intents_dict['thanks']
print(thanks)

['thank you very much', 'thanks a lot', 'thank you so much', 'thanks so much', 'thank you kindly', 'thank you for that.', 'Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me']


In [437]:
second_part_nouns = ["lifesaver", "star", "gem", "legend", "hero", "saint", "miracle worker", "godsend", "savior", "treasure"]
second_part = [" You've been a great help.", " You're a great help", " I don't know what I'd do without you."]

for noun in second_part_nouns:
    second_part.append(f" You're a {noun}.")
    second_part.append("") # for variation

In [438]:
# capitalize the first letter of each element
thanks = [word.capitalize() for word in thanks]
print(thanks)

['Thank you very much', 'Thanks a lot', 'Thank you so much', 'Thanks so much', 'Thank you kindly', 'Thank you for that.', 'Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me']


In [439]:
combinations = list(itertools.product(thanks, second_part))
combinations_joined = ['. '.join(combination) if combination[1] != "" else combination[0] for combination in combinations]
print(len(combinations_joined))

253


In [440]:
res = []
for combination in combinations_joined:
    res.append("You're very welcome. Is there anything else I can assist you with today?")

intents_dict['thanks'] = combinations_joined
response_dict['thanks'] = res

# adding options intents

In [441]:
print(intents_dict['options'])

['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered']


In [442]:
print(response_dict['options'])

['I can guide you through Adverse drug reaction list, Blood pressure tracking, Hospitals and Pharmacies', 'Offering support for Adverse drug reaction, Blood pressure, Hospitals and Pharmacies']


# Todo: make responses more varied (?)

In [443]:
options = []
responses = []
end_part = ["?", " here?", " today?", " right now?", " now?", " this evening?", " this morning?", " this afternoon?"]
help = ["help", "assist", "support", "aid"]
choices = ["options", "choices"]
response_options = ['I can guide you through the adverse drug reaction list, blood pressure tracking, or find hospitals and pharmacies.',
                    'I can offer you support by giving you information on the adverse drug reaction list, blood pressure tracking, or finding hospitals and pharmacies.',
                    'I can help you with the adverse drug reaction list, blood pressure tracking, or finding hospitals and pharmacies.']

for part in end_part:
    options.append(f"What can I do{part}")
    options.append(f"What can you do for me{part}")
    for choice in choices:
        options.append(f"What are my {choice}{part}")
    
    for help_choice in help:
        options.append(f"What can you {help_choice} me with{part}")
        options.append(f"How can you {help_choice} me{part}")
        options.append(f"What can you do to {help_choice} me{part}")

for option in options:
    responses.append(random.choice(response_options))

In [444]:
len(options)

128

Save the updated options

In [445]:
intents_dict['options'] = options
response_dict['options'] = responses

# adding adverse_drug intents

In [446]:
intents_dict['adverse_drug']

['How to check Adverse drug reaction?',
 'Open adverse drugs module',
 'Give me a list of drugs causing adverse behavior',
 'List all drugs suitable for patient with adverse reaction',
 'Which drugs dont have adverse reaction?']

In [447]:
response_dict['adverse_drug']

['Navigating to Adverse drug reaction module']

In [448]:
def generate_adverse_drug_question(number_of_questions=10):
    inquiries = []
    # data collected using code in named_entity_recognition.ipynb
    medications = pd.read_csv('../clean_data/medications.csv')
    choices = ['bad', 'adverse']
    interactions = ['interactions', 'effects', 'side effects', 'behavior']
    display = ['display', 'show', 'list', 'provide', 'give']
    med_syns = ['medications', 'drugs', 'pharmaceuticals', 'prescriptions', 'meds', 'medicines']
    time = ['at the same time?', 'simultaneously?', 'together?', 'with each other?', 'concurrently?']

    for i in range(number_of_questions):
        inquiries.append(f"Give me a list of {random.choice(med_syns)} causing {random.choice(choices)} {random.choice(interactions)}")
        inquiries.append(f"Which {random.choice(med_syns)} have {random.choice(choices)} {random.choice(interactions)}?")
        inquiries.append(f"{random.choice(display).capitalize()} all {random.choice(med_syns)} with {random.choice(choices)} {random.choice(interactions)}")
        inquiries.append(f"Is it safe to take {random.choice(medications['medication'].values)} and {random.choice(medications['medication'].values)} {random.choice(time)}")
        inquiries.append(f"Do {random.choice(medications['medication'].values)} and {random.choice(medications['medication'].values)} have {random.choice(choices)} {random.choice(interactions)}?")

    return inquiries

inquiries = generate_adverse_drug_question(20)
intents_dict['adverse_drug'] = inquiries
response_dict['adverse_drug'] = ["Navigating to Adverse drug reaction module" for inquiry in inquiries]

# adding blood-pressure intents

In [449]:
intents_dict['blood_pressure']

['Open blood pressure module',
 'Task related to blood pressure',
 'Blood pressure data entry',
 'I want to log blood pressure results',
 'Blood pressure data management']

In [450]:
response_dict['blood_pressure']

['Navigating to Blood Pressure module']

In [451]:
inquiries = []
data = ['results', 'data', 'information', 'readings', 'numbers', 'stats']
open_module = ['Open', 'Show', 'Display', 'Start', 'Run', 'Begin', 'Launch']
module = ['module', 'manager', 'system', 'app', 'tool', 'program', 'application', 'software']
log = ['log', 'record', 'track', 'read', 'view', 'monitor', 'check', 'watch', 'see']
i = 0 # for use in first loop
for elem in open_module:
    for module_syn in module:
        inquiries.append(f"{elem} the {module_syn} for blood pressure")
        inquiries.append(f"{elem} blood pressure {module_syn}")

for data_syn in data:
    for module_syn in module:
        inquiries.append(f"Blood pressure {data_syn} {module_syn}")
    
    for elem in open_module:
        inquiries.append(f"{elem} blood pressure {data_syn}")

for log_syn in log:
    inquiries.append(f"{log_syn} my blood pressure")
    inquiries.append(f"{log_syn} blood pressure")
    inquiries.append(f"I want to {log_syn} my blood pressure")

    for data_syn in data:
        inquiries.append(f"{log_syn} my blood pressure {data_syn}")
        inquiries.append(f"{log_syn} blood pressure {data_syn}")
        inquiries.append(f"I want to {log_syn} my blood pressure {data_syn}")

responses = []

for inquiry in inquiries:
    responses.append('Navigating to Blood Pressure module')

intents_dict['blood_pressure'] = inquiries
response_dict['blood_pressure'] = responses

In [452]:
len(inquiries)

364

# adding blood_pressure_search intents

In [453]:
intents_dict['blood_pressure_search']

['I want to search for blood pressure result history',
 'Blood pressure for patient',
 'Load patient blood pressure result',
 'Show blood pressure results for patient',
 'Find blood pressure results by ID']

In [454]:
response_dict['blood_pressure_search']

['Please provide Patient ID', 'Patient ID?']

In [455]:
searches = []
search_words = ['search for', 'find', 'locate', 'view', 'show', 'display', 'pull up', 'load']
result_words = ['results', 'data', 'information', 'readings', 'numbers', 'stats', 'history', 'logs', 'records']
end_part = ['for patient', 'by ID', 'for patient by ID', 'for patient by name', 'for patient by name and ID', 'for patient by name or ID', 'for patient by ID or name']
for result_word in result_words:
    for end in end_part:
        for search_word in search_words:
            searches.append(f"I want to {search_word} blood pressure {result_word} {end}")
            searches.append(f"{search_word.capitalize()} blood pressure {result_word} {end}")
        searches.append(f"Show me blood pressure {result_word} {end}")
        searches.append(f"I need {result_word} {end}")
        searches.append(f"I want {result_word} {end}")
        searches.append(f"Blood pressure {result_word} {end}")

intents_dict['blood_pressure_search'] = searches
responses = []
for search in searches:
    responses.append('Please provide patient ID')

response_dict['blood_pressure_search'] = responses

# adding search_blood_pressure_by_patient_id intents

In [456]:
intents_dict['search_blood_pressure_by_patient_id']

[]

In [457]:
response_dict['search_blood_pressure_by_patient_id']

['Loading Blood pressure result for Patient']

In [458]:
def generate_random_patient_ids(num_ids=10, prefix="P", id_length=6):
    random_ids = []
    for _ in range(num_ids):
        id_number = ''.join([str(random.randint(0, 9)) for _ in range(id_length)])
        random_ids.append(prefix + id_number)
    return random_ids

# Generate 100 random patient IDs
random_patient_ids = generate_random_patient_ids(num_ids=100)

In [459]:
searches = []
search_words = ['search for', 'find', 'locate', 'view', 'show', 'display', 'pull up', 'load']
result_words = ['results', 'data', 'information', 'readings', 'numbers', 'stats', 'history', 'logs', 'records']
for result_word in result_words:
    for search_word in search_words:
        searches.append(f"I want to {search_word} blood pressure {result_word} for patient {random.choice(random_patient_ids)}")
        searches.append(f"{search_word.capitalize()} blood pressure {result_word} for patient {random.choice(random_patient_ids)}")
        searches.append(f"{random.choice(random_patient_ids)} blood pressure {result_word}")
        searches.append(random.choice(random_patient_ids))

intents_dict['search_blood_pressure_by_patient_id'] = searches
responses = []
for search in searches:
    responses.append('Loading blood pressure data')

response_dict['search_blood_pressure_by_patient_id'] = responses

# adding pharmacy_search intents

In [460]:
intents_dict['pharmacy_search']

['Find me a pharmacy',
 'Find pharmacy',
 'List of pharmacies nearby',
 'Locate pharmacy',
 'Search pharmacy']

In [461]:
response_dict['pharmacy_search']

['Please provide pharmacy name']

In [462]:
searches = []
search_words = ['search for', 'find', 'locate', 'show', 'navigate to']
location_words = ['location', 'address', 'home', 'position']
pharmacy_words = ['pharmacy', 'drugstore', 'pharmacies', 'drugstores']
for search_word in search_words:
    searches.append(f"I want to {search_word} a pharmacy")
    searches.append(f"{search_word.capitalize()} a pharmacy")
    for pharmacy_word in pharmacy_words:
        for location_word in location_words:
            searches.append(f"{search_word.capitalize()} {pharmacy_word} near my {location_word}")

        searches.append(f"{search_word.capitalize()} {pharmacy_word}")
        searches.append(f"{search_word} {pharmacy_word} near me")
        searches.append(f"{search_word.capitalize()} nearyby {pharmacy_word}")

intents_dict['pharmacy_search'] = searches
responses = []
for search in searches:
    responses.append('Please provide pharmacy name')

response_dict['pharmacy_search'] = responses

# adding search_pharmacy_by_name intents

In [463]:
intents_dict['search_pharmacy_by_name']

[]

In [464]:
response_dict['search_pharmacy_by_name']

['Loading pharmacy details']

In [465]:
searches = []
search_words = ['search for', 'find', 'locate', 'show', 'navigate to']
names = ['CVS', 'Walgreens', 'Rite Aid', 'Walmart', 'Costco', 'Kroger', 'Publix', 'Safeway', 'Albertsons', 'Target']
for search_word in search_words:
    for name in names:
        searches.append(f"I want to {search_word} {name}")
        searches.append(f"{search_word.capitalize()} {name}")
        searches.append(f"{search_word} {name} near me")
        searches.append(f"{search_word.capitalize()} {name} nearby")
        searches.append(f"{search_word.capitalize()} {name} near my location")
intents_dict['search_pharmacy_by_name'] = searches
responses = []
for search in searches:
    responses.append('Loading pharmacy details')
    
response_dict['search_pharmacy_by_name'] = responses

# adding hospital_search intents

In [466]:
intents_dict['hospital_search']

['Lookup for hospital',
 'Searching for hospital to transfer patient',
 'I want to search hospital data',
 'Hospital lookup for patient',
 'Looking up hospital details']

In [467]:
response_dict['hospital_search']

['Please provide hospital name or location']

In [468]:
searches = []
search_words = ['search for', 'find', 'locate', 'view', 'show', 'display', 'pull up', 'load', 'lookup']
hospital = ['hospital', 'hospitals']
location_words = ['location', 'address', 'home', 'position']


for search_word in search_words:
    searches.append(f"I want to {search_word} a hospital")
    searches.append(f"{search_word.capitalize()} a hospital")
    searches.append(f"{search_word.capitalize()} {random.choice(hospital)}")
    searches.append(f"{search_word.capitalize()} {random.choice(hospital)} near me")
    searches.append(f"{search_word.capitalize()} nearyby {random.choice(hospital)}")
    for location_word in location_words:
        searches.append(f"{search_word.capitalize()} {random.choice(hospital)} near my {location_word}")

intents_dict['hospital_search'] = searches
responses = []
for search in searches:
    responses.append('Please provide hospital name, location, or type')

response_dict['hospital_search'] = responses

# adding search_hospital_by_params intents

In [469]:
intents_dict['search_hospital_by_params']

[]

In [470]:
response_dict['search_hospital_by_params']

['Please provide hospital type']

In [471]:
# list of hospital names
names = ['Mayo Clinic', 'Cleveland Clinic', 'Johns Hopkins Hospital', 'Massachusetts General Hospital',
         'UCSF Medical Center', 'UCLA Medical Center', 'New York-Presbyterian Hospital', 'Stanford Health Care-Stanford Hospital',
         'Hospitals of the University of Pennsylvania-Penn Presbyterian', 'Cedars-Sinai Medical Center', 'Northwestern Memorial Hospital',
         'UPMC Presbyterian Shadyside', 'University of Michigan Hospitals-Michigan Medicine', 'Mount Sinai Hospital']
# list of hospital addresses
addresses = ['200 1st St SW, Rochester, MN 55905', '9500 Euclid Ave, Cleveland, OH 44195', '1800 Orleans St, Baltimore, MD 21287',
             '55 Fruit St, Boston, MA 02114', '505 Parnassus Ave, San Francisco, CA 94143', '757 Westwood Plaza, Los Angeles, CA 90095',
             '525 East 68th St, New York, NY 10065', '300 Pasteur Dr, Stanford, CA 94305', '51 N 39th St, Philadelphia, PA 19104',
             '8700 Beverly Blvd, Los Angeles, CA 90048', '251 E Huron St, Chicago, IL 60611', '200 Lothrop St, Pittsburgh, PA 15213',
             '1500 E Medical Center Dr, Ann Arbor, MI 48109', '1 Gustave L Levy Pl, New York, NY 10029']

intents_dict['search_hospital_by_params'] = names + addresses

response_dict['search_hospital_by_params'] = ['Loading hospital details' for _ in range(len(intents_dict['search_hospital_by_params']))]

# adding search_hospital_by_type intents

In [472]:
# list of hospital types
types = ['general', 'community', 'teaching', 'specialty', 'clinic', 'psychiatric', 'rehabilitation', 'children', 'geriatric', 'maternity']
searches = []

for type in types:
    searches.append(f"Find me a {type} hospital")
    searches.append(f"{type} hospital")
    searches.append(f"Find me a {type} hospital near me")
    searches.append(f"{type} hospital near me")
    searches.append(f"Find me a {type} hospital near my location")
    searches.append(f"{type} hospital near my location")

intents_dict['search_hospital_by_type'] = searches
response_dict['search_hospital_by_type'] = ['Loading hospital details' for search in searches]

In [473]:
# Load the original JSON file
with open('../raw_data/intents.json', 'r') as file:
    data = json.load(file)

# Update the JSON structure with new patterns and responses except for the 'noanswer' intent
for intent in data["intents"]:
    tag = intent["tag"]
    if tag in intents_dict:
        intent["patterns"] = intents_dict[tag]
    if tag in response_dict:
        intent["responses"] = response_dict[tag]

data["intents"] = [intent for intent in data["intents"] if intent["tag"] != "noanswer"]

# Write the updated JSON structure back to a file
with open('../clean_data/intents_enriched.json', 'w') as file:
    json.dump(data, file, indent=4)